0. Load Data and Libraries

In [3]:
from pandas import read_csv
df = read_csv("C:/Users/qhaskovec/OneDrive/GitHub/PhD-Thesis-EU-Solidarity-Statements/solidarity_labels_full.csv")
df = df[["Solidarity", "context"]]
label_map = {"No Solidarity": 0, "Solidarity": 1}
df["Solidarity"] = df["Solidarity"].map(label_map)  # if not already 0/1

In [5]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

1. Train The Model

1.1 Load the Model

In [ ]:
# === Define model ID and load tokenizer + classification model ===
model_id = "EuroBERT/EuroBERT-2.1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

configuration_eurobert.py:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/EuroBERT/EuroBERT-2.1B:
- configuration_eurobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_eurobert.py:   0%|          | 0.00/43.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/EuroBERT/EuroBERT-2.1B:
- modeling_eurobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/9.61G [00:00<?, ?B/s]

1.2 Run the Model

In [12]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
# === Tokenize function ===
def tokenize_function(example):
    return tokenizer(example["context"], padding="max_length", truncation=True, max_length=128)

# === Convert to Hugging Face dataset and tokenize ===
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_function, batched=True)
#dataset = dataset.remove_columns(["context", "__index_level_0__"])  # drop unused columns if present
dataset.set_format("torch")

# === Split train/eval sets ===
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# === Define training arguments ===
training_args = TrainingArguments(
    output_dir="./bert_solidarity_finetuned",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# === Define evaluation metric ===
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.tensor(logits).argmax(dim=1)
    return accuracy.compute(predictions=predictions, references=labels)

# === Train model ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# === Save final model ===
model.save_pretrained("./bert_solidarity_finetuned")
tokenizer.save_pretrained("./bert_solidarity_finetuned")

print("Fine-tuning complete.")


Map:   0%|          | 0/6904 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
